<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.7

In [7]:
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import io
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers
from sklearn.datasets import load_breast_cancer, load_boston

#class instantiations
ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

def BigDawg(X, y, params, task, ts=150, epochs=50, batch_size=32, patience=5, regression=False):
  '''facilitates the Data Collection Process'''
  dl = DeepLearning()
  wr = Wrappers()
  if task == 'NLP':
    X2, y2, _ = wr.TCP(X, y)
    func = dl.RNN #insert CV after this
  if task == 'CV':
    X2, _, y2, _ = dl.ModelReadyPixles(X, y, target_size=(ts, ts))
    func = dl.CNN
  if task == 'TC':
    X2 = np.array(X)
    y2 = dl.MulticlassOutput(np.array(y))
    func = dl.DeepTabularClassification
  if task == 'TR':
    X2 = np.array(X)
    y2 = np.array(y)
    func = dl.DeepTabularRegression
  try:
    return dl.CollectPerformance(params, func, X2, y2, epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)
  except:
    return dl.CollectPerformance(params, func, X2, y2.reshape(-1), epochs=epochs, batch_size=batch_size, patience=patience, regression=regression)

In [8]:
#Dataframes
Wikipedia = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Wikipedia.csv') #text
Twitter = pd.read_csv('https://raw.githubusercontent.com/spe301/AI-generated-AI/main/Data/Twitter.csv') #text

#Series
articles = Wikipedia['Text']
labels = Wikipedia['AI']
tweets = Twitter['tweet']
sentiments = Twitter['is_there_an_emotion_directed_at_a_brand_or_product']

#arrays
cancer_data = load_breast_cancer()['data']
diagnosis = load_breast_cancer()['target']
housing_data = load_boston()['data']
prices = load_boston()['target']

#images
#uploaded = files.upload()
#data = zipfile.ZipFile(io.BytesIO(uploaded['IMGDS1.zip']), 'r')
#data.extractall()

In [4]:
#image folder directories, if using image data

In [12]:
X = housing_data
y = prices

#output = len(np.unique(y))
#embedding = dh.GetVocab(df, target_str)
#rnn_grid = {'output_dim': [output], 'embedding': [embedding], 'nodes': [64], 'activation': ['tanh'], 'regularizer': [None], 
            #'stacking': [False], 'dropout': [False], 'optimizer': ['adam'], 'method': ['GRU'], 'bidirectional': [False]}

#output = len(y)
#cnn_grid = {'output_dim': [output], 'base_filters':[32], 'kernel_size': [2], 'activation': ['relu'], 'nblocks': [3], 
            #'pool':[2], 'dropout': [False], 'closer': [False], 'optimizer': ['adam'], 'metrics': ['accuracy']}

#output = len(np.unique(y))
#tc_grid = {'output_dim': [output], 'nodes': [64], 'activation': ['relu'], 'regularizer': [None], 'stacking': [False], 
           #'dropout': [False], 'nlayers': [4], 'closer': [False], 'loss_func': ['binary_crossentropy'], 'optimizer': ['adam']}

#tr_grid = {'nodes': [64], 'activation': ['relu'], 'regularizer': [None], 'stacking': [False], 'dropout': [False], 
           #'nlayers': [4], 'closer': [False], 'loss_func': ['MSE'], 'optimizer': ['rmsprop'], 'y_col': ['string']}

BigDawg(X, y, tr_grid, 'TR', epochs=3, regression=True)

preforming 4 total fits ...
Epoch 1/3
13/13 [==============================] - 1s 25ms/step - loss: 346.5579 - val_loss: 71.4274
Epoch 2/3
13/13 [==============================] - 0s 4ms/step - loss: 103.5362 - val_loss: 41.5829
Epoch 3/3
13/13 [==============================] - 0s 4ms/step - loss: 76.3094 - val_loss: 38.5488
1 of 4 fits complete!
Epoch 1/3
13/13 [==============================] - 1s 12ms/step - loss: 948.0668 - val_loss: 156.0159
Epoch 2/3
13/13 [==============================] - 0s 4ms/step - loss: 100.3612 - val_loss: 308.1322
Epoch 3/3
13/13 [==============================] - 0s 4ms/step - loss: 152.8413 - val_loss: 75.7847
2 of 4 fits complete!
Epoch 1/3
13/13 [==============================] - 1s 12ms/step - loss: 187.2464 - val_loss: 28.6337
Epoch 2/3
13/13 [==============================] - 0s 4ms/step - loss: 77.7607 - val_loss: 34.5866
Epoch 3/3
13/13 [==============================] - 0s 4ms/step - loss: 88.1901 - val_loss: 171.9215
3 of 4 fits complete!
Epo

,nodes,activation,regularizer,stacking,dropout,nlayers,closer,loss_func,y_col,epochs,batch_size,optimizer,loss,val_loss
0,64,relu,None,False,False,4,False,MSE,price,3,32,rmsprop,72.766449,38.548794
1,64,relu,None,False,False,4,False,MAE,price,3,32,rmsprop,146.360703,75.784737
2,64,relu,L1,False,False,4,False,MSE,price,3,32,rmsprop,77.046013,171.921524
3,64,relu,L1,False,False,4,False,MAE,price,3,32,rmsprop,99.578064,45.160343


# Image dataset directory graveyard
copy and paste these into a code cell when working with the dataset

Dogs and Cats
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']